In [ ]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt

In [ ]:
folder_path = r"/storage/alplakes_test/lucerne_100m_2025/outputs_swirl"
input_folder = os.path.join(folder_path, "eddy_catalogues_lvl0")

output_folder = os.path.join(folder_path, "ke_eddy")
os.makedirs(output_folder, exist_ok=True)

In [ ]:
lvl0_csv_path = os.path.join(input_folder, "lvl0_20250101_20251231_concat.csv")
lake_csv_path = os.path.join(input_folder, "lake_characteristics_20250101_20251231_concat.csv")

In [ ]:
df_lvl0 = pd.read_csv(lvl0_csv_path)
df_lvl0 = df_lvl0.set_index('id', drop=False)

In [ ]:
df_lake = pd.read_csv(lake_csv_path)
df_lake = df_lake.set_index('id', drop=False)

# Entire lake analysis

In [ ]:
eddy_ke = df_lvl0.groupby('date').sum()['kinetic_energy_eddy_[MJ]']

In [ ]:
lake_ke = df_lake.groupby('date').sum()['kinetic_energy_[MJ]']

In [ ]:
eddy_ke.plot()

In [ ]:
eddy_ke.reset_index().to_csv(os.path.join(output_folder, "ke_eddies.csv"))

In [ ]:
lake_ke.reset_index().to_csv(os.path.join(output_folder, "ke_lake.csv"))

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1, 1, 1)
lake_ke.plot(label='Entire lake')
eddy_ke.plot(label='Eddies')
plt.ylabel('Kinetic Energy [MJ]')
plt.xlabel('')
plt.xticks(rotation=10)
plt.legend()
#ax.set_yscale('log')
fig.savefig(os.path.join(output_folder, "kinetic_energy_whole_lake.png"))

In [ ]:
percentage_total_ke = 100 * eddy_ke / lake_ke[lake_ke.index.isin(eddy_ke.index)]

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1, 1, 1)
percentage_total_ke.plot(label='Entire lake')
plt.title('Fraction of total kinetic energy contained in eddies')
plt.ylabel('Fraction [%]')
plt.xlabel('')
plt.xticks(rotation=10)
plt.legend()

# Surface analysis

In [ ]:
df_lvl0_surf = df_lvl0[df_lvl0['depth_index']==0]
df_lake_surf = df_lake[df_lake['depth_index']==0]

In [ ]:
surf_lvl0_sum = df_lvl0_surf.groupby('date').sum()

surf_lvl0_sum['kinetic_energy_eddy_[MJ]'].reset_index().to_csv(r'D:\04-Geneva_dummy\03_24h-wind_extended\energy_analysis\surface_eddy_ke')

df_lake_surf[df_lake_surf.index.isin(surf_lvl0_sum.index)]['kinetic_energy_[MJ]'].reset_index().to_csv(r'D:\04-Geneva_dummy\03_24h-wind_extended\energy_analysis\ke_total_surface.csv')

In [ ]:
surf_lvl0_sum['percentage_total_ke_[%]'] = 100 * surf_lvl0_sum['kinetic_energy_eddy_[MJ]'] / df_lake_surf[df_lake_surf.index.isin(surf_lvl0_sum.index)]['kinetic_energy_[MJ]']

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1, 1, 1)
percentage_total_ke.plot(label='Entire lake')
surf_lvl0_sum['percentage_total_ke_[%]'].plot(label='Lake surface')
plt.title('Fraction of total kinetic energy contained in eddies')
plt.ylabel('Fraction [%]')
plt.xlabel('')
plt.xticks(rotation=10)
plt.legend()
fig.savefig(os.path.join(output_folder, "fraction_ke_in_eddies.png"))

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1, 1, 1)
df_lake_surf['kinetic_energy_[MJ]'].plot(label='Total')
surf_lvl0_sum['kinetic_energy_eddy_[MJ]'].plot(label='Eddies')
plt.legend()
plt.ylabel('Kinetic Energy surface [MJ]')
plt.xlabel('')
plt.xticks(rotation=10)
#ax.set_yscale('log')
fig.savefig(os.path.join(output_folder, "kinetic_energy_surface.png"))

In [ ]:
df_lake_surf['kinetic_energy_[MJ]'].plot(label='Entire lake')

In [ ]:
surf_lvl0_sum['kinetic_energy_eddy_[MJ]'].plot()

In [ ]:
surf_lvl0_sum['surface_area_[m2]'].plot()
df_lake_surf['surface_area_[m2]'].plot()

# Depth analysis

In [ ]:
eddy_ke_by_depth_and_date = df_lvl0.groupby(['time_index', 'depth_[m]'])['kinetic_energy_eddy_[MJ]'].sum().reset_index()

In [ ]:
t_idx = 0
fig, ax = plt.subplots(figsize=(5,7))

# Lake KE profile
df_lake[df_lake['time_index']==t_idx].plot(
    x='kinetic_energy_[MJ]', y='depth_[m]', ax=ax, label='Lake KE'
)

# Eddy KE profile
eddy_ke_by_depth_and_date[eddy_ke_by_depth_and_date['time_index']==t_idx].plot(
    x='kinetic_energy_eddy_[MJ]', y='depth_[m]', ax=ax, label='Eddy KE'
)

ax.set_xlabel("Kinetic Energy [MJ]")
ax.set_ylabel("Depth [m]")
ax.legend()
plt.title(df_lake[df_lake['time_index']==t_idx].index[0])
ax.set_xlim(left=0, right=5000)
plt.show()
str_time = df_lake[df_lake['time_index']==t_idx].index[0].replace(' ','_').replace(':','-')
fig.savefig(os.path.join(output_folder, "profiles_ke", f"kinetic_energy_profile_{str_time}.png"))